In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('ratings.dat',sep = '::',header = None)
movies = pd.read_csv('movies.dat',sep = '::',header = None)
users = pd.read_csv('users.dat',sep = '::',header = None)
df.columns=['userId','movieId','Rating','timestamp']
movies.columns = ['movieId','name','tag']
users.columns = ['userId','sex','age','occupation','zipcode']
df1 = pd.merge(df,movies, on = 'movieId')
data = pd.merge(df1,users,on = 'userId')

In [4]:
data.head()

,userId,movieId,Rating,timestamp,name,tag,sex,age,occupation,zipcode
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [5]:
data.columns

Index(['userId', 'movieId', 'Rating', 'timestamp', 'name', 'tag', 'sex', 'age',
       'occupation', 'zipcode'],
      dtype='object')

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [7]:
###对特征标签进行编码
sparse_features = ['userId', 'movieId','sex', 'age','occupation', 'zipcode']
target = ['Rating']
for feature in sparse_features:
    lbe  = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [8]:
from deepctr.models import WDL
from deepctr.inputs import SparseFeat,get_feature_names

In [9]:
fixlen = [SparseFeat(feature,data[feature].nunique()) for feature in sparse_features]

In [10]:
linear = fixlen
dnn = fixlen

In [11]:
feature_names = get_feature_names(linear + dnn)

In [12]:
train,test  = train_test_split(data,test_size = 0.3)
train_model_input  = {name:train[name].values for name in feature_names}
test_model_input  = {name:test[name].values for name in feature_names}

In [13]:
model = WDL(linear,dnn,task='regression')
model.compile('adam','mse',metrics = ['mse'])
history = model.fit(train_model_input,train[target].values,batch_size = 256,epochs=3,verbose = True,validation_split = 0.2)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 560116 samples, validate on 140030 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
560116/560116 [==============================] - 27s 48us/sample - loss: 1.0375 - mean_squared_error: 1.0369 - val_loss: 0.8308 - val_mean_squared_error: 0.8297
Epoch 2/3
560116/560116 [==============================] - 23s 40us/sample - loss: 0.8141 - mean_squared_error: 0.8125 - val_loss: 0.8119 - val_mean_squared_error: 0.8099
Epoch 3/3
560116/560116 [==============================] - 23s 41us/sample - loss: 0.7922 - mean_squared_error: 0.7896 - val_loss: 0.8006 - val_mean_squared_error: 0.7975


In [14]:
pred = model.predict(test_model_input,batch_size=256)
pred

array([[3.49645  ],
       [3.1164985],
       [4.171897 ],
       ...,
       [3.4926143],
       [3.0304494],
       [4.2251287]], dtype=float32)

In [15]:
mse = round(mean_squared_error(test[target].values,pred),4)
rmse = mse**0.5
print('RMSE:',rmse)

RMSE: 0.8915155635209068
